In [1]:
import pandas as pd
import os

print(os.getcwd())

C:\Users\user\Desktop\지금


# 주소->좌표변환

In [6]:
address_target =pd.read_csv("00.data.csv")
address_target


,Unnamed: 0,접수일시,cctv관리번호,주소,영상 요청 시작일,영상 요청 종료일,목적,사유,비고,위도,경도
0,0,2020-10-28,교-025,경기도 구리시 교문동 754-3,2020-10-20 19:10,2020-10-20 20:10,기타(수사목적),교통사고로 인한 영상,도림초교앞주유소01,37.595119,127.131562
1,1,2020-10-27,토-068,경기도 구리시 수택동 684-1,2020-10-25 11:10,2020-10-25 17:10,교통조사,교통사고 조사,새마을공원,37.593117,127.148706
2,2,2020-10-27,갈-024,경기도 구리시 갈매동 303-3,2020-10-18 00:10,2020-10-18 23:10,교통조사,강제추행관련 수사,아름터프라자,37.631667,127.117246
3,3,2020-10-27,갈-038,경기도 구리시 갈매동 296-1,2020-10-18 16:10,2020-10-18 18:10,성폭력,강제추행 관련 범죄수사,중앙공원무대-갈매천변,37.632644,127.118980
4,4,2020-10-27,인-052,경기도 구리시 인창동 619-12,2020-09-14 14:09,2020-09-14 16:09,기타(수사목적),경범죄처벌법위반 피의자의 소재를 파악하기위해,인창초후문,37.604070,127.130835
...,...,...,...,...,...,...,...,...,...,...,...
1069,1069,2020-02-14,인-066,경기도 구리시 사노동 360-5,2020-02-14 10:02,2020-02-14 10:02,절도,수사,연세마음 병원앞,37.637305,127.141279
1070,1070,2020-02-14,수-005,경기도 구리시 수택동 521-13,2020-02-09 23:02,2020-02-10 08:02,절도,절도사건 수사,구리세탁소04,37.600212,127.144243
1071,1071,2020-02-14,토-090,경기도 구리시 수택동 875,2020-02-11 00:02,2020-02-11 00:02,절도,절도 피의자 추적수사,수택3동주민센터 뒤01,37.586747,127.138736
1072,1072,2020-02-13,토-050,경기도 구리시 수택동 441-12,2020-02-06 20:02,2020-02-07 08:02,교통조사,사고내용 조사를 위해,"신래주택,경복방앗간 앞02",37.593533,127.140209


In [7]:
address_target = address_target[['접수일시', 'cctv관리번호', '주소', '목적', '사유']]

In [8]:
#주소변환
import requests
import urllib.request
import json
import time
from pandas import DataFrame
import pandas as pd


# address_target =pd.read_csv("C:/Users/user/Desktop/geocoding_sample1.csv", engine='python')

#쉼표, 괄호 삭제
#X, Y, region_1depth_name, region_2depth_name, region_3depth_name
Information = {}
address_list = []
X_list = []
Y_list = []
region_1depth = []
region_2depth = []
region_3depth = []
cnt=1

for i in address_target['주소']:

    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers =  {'Authorization': 'KakaoAK bf51b69fcfb397887c85f7fa1eb7460d'}
    data = {'query': i}
    req = requests.get(url, headers=headers, data=data)
    #time.sleep(3)
    req.get_method = lambda: "GET"
    response_body = req.json()
    req.close
    
    text = response_body['documents']
#     print(text)

    try:
        XY = dict(text[0])
    except IndexError:
        X_list.append('0')
        Y_list.append('0')
        region_1depth.append('0')
        region_2depth.append('0')
        region_3depth.append('0')
        address_list.append(i)
    else:
        if XY['road_address'] == None:
            X_list.append(XY['x'])
            Y_list.append(XY['y'])
            region_1depth.append(XY['address']['region_1depth_name'])
            region_2depth.append(XY['address']['region_2depth_name'])
            region_3depth.append(XY['address']['region_3depth_name'])
            address_list.append(i)
            
        elif XY['address'] == None:
            X_list.append(XY['x'])
            Y_list.append(XY['y'])
            region_1depth.append(XY['road_address']['region_1depth_name'])
            region_2depth.append(XY['road_address']['region_2depth_name'])
            region_3depth.append(XY['road_address']['region_3depth_name'])
            address_list.append(i)
        else:
            X_list.append(XY['x'])
            Y_list.append(XY['y'])
            region_1depth.append(XY['address']['region_1depth_name'])
            region_2depth.append(XY['address']['region_2depth_name'])
            region_3depth.append(XY['address']['region_3depth_name'])
            address_list.append(i)

            


address_target['address'] = address_list
address_target['X'] = X_list
address_target['Y'] = Y_list
address_target['region_1depth'] = region_1depth
address_target['region_2depth'] = region_2depth
address_target['region_3depth'] = region_3depth


address_target.to_csv("api_좌표.csv", encoding = 'euc-kr')
  

<ipython-input-8-13661619c1ce>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_target['address'] = address_list
<ipython-input-8-13661619c1ce>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_target['X'] = X_list
<ipython-input-8-13661619c1ce>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [9]:
address_target

,접수일시,cctv관리번호,주소,목적,사유,address,X,Y,region_1depth,region_2depth,region_3depth
0,2020-10-28,교-025,경기도 구리시 교문동 754-3,기타(수사목적),교통사고로 인한 영상,경기도 구리시 교문동 754-3,127.131438508727,37.5951965984903,경기,구리시,교문동
1,2020-10-27,토-068,경기도 구리시 수택동 684-1,교통조사,교통사고 조사,경기도 구리시 수택동 684-1,127.148725896357,37.5931104692712,경기,구리시,수택동
2,2020-10-27,갈-024,경기도 구리시 갈매동 303-3,교통조사,강제추행관련 수사,경기도 구리시 갈매동 303-3,0,0,0,0,0
3,2020-10-27,갈-038,경기도 구리시 갈매동 296-1,성폭력,강제추행 관련 범죄수사,경기도 구리시 갈매동 296-1,0,0,0,0,0
4,2020-10-27,인-052,경기도 구리시 인창동 619-12,기타(수사목적),경범죄처벌법위반 피의자의 소재를 파악하기위해,경기도 구리시 인창동 619-12,127.13084668944,37.6040849845449,경기,구리시,인창동
...,...,...,...,...,...,...,...,...,...,...,...
1069,2020-02-14,인-066,경기도 구리시 사노동 360-5,절도,수사,경기도 구리시 사노동 360-5,127.141291784849,37.6372993588126,경기,구리시,사노동
1070,2020-02-14,수-005,경기도 구리시 수택동 521-13,절도,절도사건 수사,경기도 구리시 수택동 521-13,127.144263418954,37.6002286078576,경기,구리시,수택동
1071,2020-02-14,토-090,경기도 구리시 수택동 875,절도,절도 피의자 추적수사,경기도 구리시 수택동 875,127.138816219146,37.5868367715326,경기,구리시,수택동
1072,2020-02-13,토-050,경기도 구리시 수택동 441-12,교통조사,사고내용 조사를 위해,경기도 구리시 수택동 441-12,127.140221291602,37.5935372271268,경기,구리시,수택동


In [10]:
pd.read_csv('00.data.csv')

,Unnamed: 0,접수일시,cctv관리번호,주소,영상 요청 시작일,영상 요청 종료일,목적,사유,비고,위도,경도
0,0,2020-10-28,교-025,경기도 구리시 교문동 754-3,2020-10-20 19:10,2020-10-20 20:10,기타(수사목적),교통사고로 인한 영상,도림초교앞주유소01,37.595119,127.131562
1,1,2020-10-27,토-068,경기도 구리시 수택동 684-1,2020-10-25 11:10,2020-10-25 17:10,교통조사,교통사고 조사,새마을공원,37.593117,127.148706
2,2,2020-10-27,갈-024,경기도 구리시 갈매동 303-3,2020-10-18 00:10,2020-10-18 23:10,교통조사,강제추행관련 수사,아름터프라자,37.631667,127.117246
3,3,2020-10-27,갈-038,경기도 구리시 갈매동 296-1,2020-10-18 16:10,2020-10-18 18:10,성폭력,강제추행 관련 범죄수사,중앙공원무대-갈매천변,37.632644,127.118980
4,4,2020-10-27,인-052,경기도 구리시 인창동 619-12,2020-09-14 14:09,2020-09-14 16:09,기타(수사목적),경범죄처벌법위반 피의자의 소재를 파악하기위해,인창초후문,37.604070,127.130835
...,...,...,...,...,...,...,...,...,...,...,...
1069,1069,2020-02-14,인-066,경기도 구리시 사노동 360-5,2020-02-14 10:02,2020-02-14 10:02,절도,수사,연세마음 병원앞,37.637305,127.141279
1070,1070,2020-02-14,수-005,경기도 구리시 수택동 521-13,2020-02-09 23:02,2020-02-10 08:02,절도,절도사건 수사,구리세탁소04,37.600212,127.144243
1071,1071,2020-02-14,토-090,경기도 구리시 수택동 875,2020-02-11 00:02,2020-02-11 00:02,절도,절도 피의자 추적수사,수택3동주민센터 뒤01,37.586747,127.138736
1072,1072,2020-02-13,토-050,경기도 구리시 수택동 441-12,2020-02-06 20:02,2020-02-07 08:02,교통조사,사고내용 조사를 위해,"신래주택,경복방앗간 앞02",37.593533,127.140209


# 주소명칭 -> 좌표

In [ ]:
#주소변환
import requests
import urllib.request
import json
import time
from pandas import DataFrame
import pandas as pd

address_target=pd.read_csv('C:/Users/user/Desktop/파일명.csv',engine='python')

#X, Y, region_1depth_name, region_2depth_name, region_3depth_name
Information = {}
address_list = []
X_list = []
Y_list = []
region_1depth = []

cnt=1


for i in address_target['주소']:
# for i in a:

    url = "https://dapi.kakao.com/v2/local/search/keyword.json?page=1&size=1&sort=accuracy"
    headers =  {'Authorization': 'KakaoAK 398962f090f64571022ed42e0fdccfcb'}
    data = {'query': i}
    req = requests.get(url, headers=headers, data=data)
    #time.sleep(3)
    req.get_method = lambda: "GET"
    response_body = req.json()
    #print(data)
    req.close
    
    text = response_body['documents']

    try:
        XY = dict(text[0])
    except IndexError:
        X_list.append('0')
        Y_list.append('0')
        region_1depth.append('0')

        address_list.append(i)
    else:
        X_list.append(XY['x'])
        Y_list.append(XY['y'])
        region_1depth.append(XY['address_name'])

        address_list.append(i)


    print(str(cnt)+'/'+str(len(address_target)))
#     print(Information)
    cnt=cnt+1

address_target['주소'] = address_list
address_target['X'] = X_list
address_target['Y'] = Y_list



address_target.to_csv("C:/Users/user/Desktop/좌표.csv", encoding = 'euc-kr')
  